In [70]:
import pandas as pd

n1_df = pd.read_csv('N1.csv')
n1_df

,N1,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,q1,q2,q3,q4
1,0,1,NaN,NaN
2,q1,NaN,NaN,NaN
3,*q4,NaN,NaN,NaN
4,q1,0,q1,NaN
5,q1,1,q1,NaN
6,q1,1,q2,NaN
7,q2,0,q3,NaN
8,q2,~,q3,NaN
9,q3,1,*q4,NaN


In [71]:
def parse_nfa(df):
    """
    Parse the NFA from the given dataframe.
    
    Args:
    - df (pd.DataFrame): DataFrame containing NFA description.
    
    Returns:
    - dict: A dictionary containing NFA components (states, input_alphabet, initial_state, final_states, transitions).
    """
    # Extracting NFA components
    states = df.iloc[0, :].dropna().tolist()
    input_alphabet = df.iloc[1, :].dropna().tolist()
    initial_state = df.iloc[2, 0]
    final_states = [state.replace('*', '') for state in df.iloc[3, :].dropna().tolist()]
    
    # Extracting transitions
    transitions_df = df.iloc[4:, :]
    transitions = []
    for _, row in transitions_df.iterrows():
        current_state, input_symbol, next_state = row[0], row[1], row[2]
        transitions.append((current_state.replace('*', ''), input_symbol, next_state.replace('*', '')))
    
    return {
        "states": states,
        "input_alphabet": input_alphabet,
        "initial_state": initial_state,
        "final_states": final_states,
        "transitions": transitions
    }

nfa = parse_nfa(n1_df)
nfa


/tmp/ipykernel_35158/3279387017.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_state, input_symbol, next_state = row[0], row[1], row[2]


{'states': ['q1', 'q2', 'q3', 'q4'],
 'input_alphabet': ['0', '1'],
 'initial_state': 'q1',
 'final_states': ['q4'],
 'transitions': [('q1', '0', 'q1'),
  ('q1', '1', 'q1'),
  ('q1', '1', 'q2'),
  ('q2', '0', 'q3'),
  ('q2', '~', 'q3'),
  ('q3', '1', 'q4'),
  ('q4', '0', 'q4'),
  ('q4', '1', 'q4')]}

In [72]:
class NFA:
    def __init__(self, states, input_alphabet, initial_state, final_states, transitions):
        self.states = states
        self.input_alphabet = input_alphabet
        self.initial_state = initial_state
        self.final_states = final_states
        self.transitions = self.build_transitions(transitions)

    def build_transitions(self, transitions):
        trans_dict = defaultdict(list)
        for (src, symbol, dest) in transitions:
            trans_dict[(src, symbol)].append(dest)
        return trans_dict

    def epsilon_closure(self, states):
        closure = set(states)
        stack = list(states)
        while stack:
            state = stack.pop()
            for next_state in self.transitions.get((state, '~'), []):
                if next_state not in closure:
                    closure.add(next_state)
                    stack.append(next_state)
        return closure

    def move(self, states, symbol):
        next_states = set()
        for state in states:
            next_states.update(self.transitions.get((state, symbol), []))
        return self.epsilon_closure(next_states)

    def paths(self, input_string):
        current_paths = [[self.initial_state]]
        for symbol in input_string:
            new_paths = []
            for path in current_paths:
                last_state = path[-1]
                next_states = self.move({last_state}, symbol)
                for state in next_states:
                    new_paths.append(path + [state])
            current_paths = new_paths

        # Filter paths that end in a final state
        valid_paths = [path for path in current_paths if path[-1] in self.final_states]
        return valid_paths

# Testing with the given example


In [73]:
nfa = NFA(**nfa)
all_paths = list(nfa.paths('0110'))

all_paths


[['q1', 'q1', 'q3', 'q4', 'q4']]